# #5 Movie Metrics

In [1]:
import polars as pl

In [2]:
users = pl.read_csv(
	'user_activity/users.csv', 
	try_parse_dates=True, 
	new_columns=['user_id']
)

activity = pl.read_csv(
	'user_activity/activity.csv', 
	try_parse_dates=True, 
	columns=['user_id', 'date', 'movie_name', 'finished']
)

In [3]:
user_act = users.join(activity,on='user_id')
user_act

user_id,created_at,country_code,date,movie_name,finished
i64,date,str,date,str,i64
2,2023-06-15,"""CA""",2023-06-22,"""The Shawshank Redemption""",1
1,2023-05-26,"""CA""",2023-07-23,"""Shrek""",0
4,2023-07-27,"""CA""",2023-07-27,"""Fight Club""",1
1,2023-05-26,"""CA""",2023-08-23,"""Top Gun: Maverick""",0
4,2023-07-27,"""CA""",2023-08-24,"""Oppenheimer""",0
…,…,…,…,…,…
20,2024-11-23,"""MX""",2025-05-14,"""Coco""",0
21,2025-01-01,"""US""",2025-05-14,"""Forrest Gump""",1
13,2024-07-02,"""MX""",2025-05-16,"""Soul""",1


In [4]:
user_act.group_by('user_id', maintain_order=True).agg(pl.min("date"))

user_id,date
i64,date
2,2023-06-22
1,2023-07-23
4,2023-07-27
5,2023-09-07
3,2023-10-05
…,…
21,2025-01-03
22,2025-01-14
23,2025-01-29


In [5]:
user_act.group_by('user_id', maintain_order=True).all()

user_id,created_at,country_code,date,movie_name,finished
i64,list[date],list[str],list[date],list[str],list[i64]
2,"[2023-06-15, 2023-06-15, … 2023-06-15]","[""CA"", ""CA"", … ""CA""]","[2023-06-22, 2023-11-01, … 2025-05-01]","[""The Shawshank Redemption"", ""Get Out"", … ""Fight Club""]","[1, 0, … 1]"
1,"[2023-05-26, 2023-05-26, … 2023-05-26]","[""CA"", ""CA"", … ""CA""]","[2023-07-23, 2023-08-23, … 2025-03-26]","[""Shrek"", ""Top Gun: Maverick"", … ""Her""]","[0, 0, … 1]"
4,"[2023-07-27, 2023-07-27, … 2023-07-27]","[""CA"", ""CA"", … ""CA""]","[2023-07-27, 2023-08-24, … 2025-05-09]","[""Fight Club"", ""Oppenheimer"", … ""Avengers: Endgame""]","[1, 0, … 1]"
5,"[2023-09-01, 2023-09-01, … 2023-09-01]","[""US"", ""US"", … ""US""]","[2023-09-07, 2023-09-25, … 2025-03-14]","[""Top Gun: Maverick"", ""Frozen"", … ""Bohemian Rhapsody""]","[1, 1, … 1]"
3,"[2023-07-18, 2023-07-18, … 2023-07-18]","[""MX"", ""MX"", … ""MX""]","[2023-10-05, 2023-11-10, … 2025-03-31]","[""The Grand Budapest Hotel"", ""Oppenheimer"", … ""Nope""]","[0, 1, … 1]"
…,…,…,…,…,…
21,"[2025-01-01, 2025-01-01, … 2025-01-01]","[""US"", ""US"", … ""US""]","[2025-01-03, 2025-01-06, … 2025-05-14]","[""Up"", ""The Revenant"", … ""Forrest Gump""]","[1, 1, … 1]"
22,"[2025-01-13, 2025-01-13, … 2025-01-13]","[""MX"", ""MX"", … ""MX""]","[2025-01-14, 2025-01-20, … 2025-05-13]","[""Knives Out"", ""Coco"", … ""Big Hero 6""]","[1, 0, … 1]"
23,"[2025-01-28, 2025-01-28, … 2025-01-28]","[""US"", ""US"", … ""US""]","[2025-01-29, 2025-01-31, … 2025-05-04]","[""Knives Out"", ""Gladiator"", … ""Mad Max: Fury Road""]","[0, 1, … 1]"


In [6]:
user_act.select('user_id', 'date', 'movie_name')

user_id,date,movie_name
i64,date,str
2,2023-06-22,"""The Shawshank Redemption"""
1,2023-07-23,"""Shrek"""
4,2023-07-27,"""Fight Club"""
1,2023-08-23,"""Top Gun: Maverick"""
4,2023-08-24,"""Oppenheimer"""
…,…,…
20,2025-05-14,"""Coco"""
21,2025-05-14,"""Forrest Gump"""
13,2025-05-16,"""Soul"""
